# Merge and interpolate with generic data

In [21]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from datetime import timedelta

Format data like below.

'Merge' column must be named equally! (Here: 'Time')

In [22]:
df1 = pd.DataFrame({'Time':   [0,2,4,6,8,10,12,14,16,18],
                    'Current':[5,6,7,6,5,4,5,4,5,6]
                   })

df2 = pd.DataFrame({'Time':   [1,3,4,6,7,11,12,14,17,19],
                    'Voltage':[9,9,7,6,5,7,6,9,6,7]  
                   })

df3 = pd.DataFrame({'Time':   [1,2,4,8,9,10,11,12,17,20],
                    'Temperature':[105,112,115,105,107,110,111,113,115,110]  
                   })

Check data:

In [23]:
print(pd.concat([df1, df2,df3]).sort_values('Time').reset_index(drop=True))

    Time  Current  Voltage  Temperature
0      0      5.0      NaN          NaN
1      1      NaN      NaN        105.0
2      1      NaN      9.0          NaN
3      2      6.0      NaN          NaN
4      2      NaN      NaN        112.0
5      3      NaN      9.0          NaN
6      4      7.0      NaN          NaN
7      4      NaN      NaN        115.0
8      4      NaN      7.0          NaN
9      6      6.0      NaN          NaN
10     6      NaN      6.0          NaN
11     7      NaN      5.0          NaN
12     8      5.0      NaN          NaN
13     8      NaN      NaN        105.0
14     9      NaN      NaN        107.0
15    10      NaN      NaN        110.0
16    10      4.0      NaN          NaN
17    11      NaN      7.0          NaN
18    11      NaN      NaN        111.0
19    12      5.0      NaN          NaN
20    12      NaN      NaN        113.0
21    12      NaN      6.0          NaN
22    14      4.0      NaN          NaN
23    14      NaN      9.0          NaN


Enter dataframe to 'merge_list'.

Define wich column to merge on.

In [24]:
merge_list = [df1,df2,df3]
merge_on   = 'Time'

In [25]:
def merger(merge_list,merge_on,data=[]):

    merge1 = merge_list[0].merge(merge_list[1], how='left', on=merge_on)
    merge2 = merge_list[1].merge(merge_list[0], how='left', on=merge_on)

    data = pd.concat([merge1,merge2]).sort_values(merge_on)
    data = data.drop_duplicates(subset=[merge_on]).reset_index(drop=True)

    if len(merge_list) > 2:
        for extra_merge in range(2,len(merge_list)):
            merge1 = data.merge(merge_list[extra_merge], how='left', on=merge_on)
            merge2 = merge_list[extra_merge].merge(data, how='left', on=merge_on)

            data = pd.concat([merge1,merge2]).sort_values(merge_on)
            data = data.drop_duplicates(subset=[merge_on]).reset_index(drop=True)
    
    return data
    
data = merger(merge_list,merge_on)
data

,Time,Current,Voltage,Temperature
0,0,5.0,NaN,NaN
1,1,NaN,9.0,105.0
2,2,6.0,NaN,112.0
3,3,NaN,9.0,NaN
4,4,7.0,7.0,115.0
5,6,6.0,6.0,NaN
6,7,NaN,5.0,NaN
7,8,5.0,NaN,105.0
8,9,NaN,NaN,107.0
9,10,4.0,NaN,110.0


Three different interploation options:

In [26]:
data_int1 = data.interpolate(limit=2, limit_direction='forward')
data_int2 = data.interpolate(limit=2, limit_direction='backward')
data_int3  = data.interpolate(method='spline', order=1, limit=10, limit_direction='both')

data_int3

,Time,Current,Voltage,Temperature
0,0,5.000000,9.822825,99.452616
1,1,5.734043,9.000000,105.000000
2,2,6.000000,8.434394,112.000000
3,3,5.606383,9.000000,113.080524
4,4,7.000000,7.000000,115.000000
5,6,6.000000,6.000000,111.564554
6,7,5.414894,5.000000,108.489603
7,8,5.000000,6.513763,105.000000
8,9,5.287234,6.594771,107.000000
9,10,4.000000,6.675779,110.000000


In [27]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=data['Time'],y=data['Voltage'],name='original',
                         mode='markers',marker=dict(size=10,color='black')))
fig.add_trace(go.Scatter(x=data_int1['Time'],y=data_int1['Voltage'],name='forward',
                         mode='markers+lines',marker=dict(size=10,symbol='circle-open',color='blue')))
fig.add_trace(go.Scatter(x=data_int2['Time'],y=data_int2['Voltage'],name='backward',
                         mode='markers+lines',marker=dict(size=10,symbol='circle-open',color='green')))
fig.add_trace(go.Scatter(x=data_int3['Time'],y=data_int3['Voltage'],name='spline',
                         mode='markers+lines',marker=dict(size=10,symbol='circle-open',color='red')))
fig.show()

# Merge and interpolate with real data

In [28]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from datetime import timedelta

Dataset #1

In [29]:
df1 = pd.read_csv('data/2020-11-09_VOL_R-80.csv').iloc[:,:2]

df1['Time'] = pd.to_datetime(df1['Time'], unit='ms') + timedelta(hours=1)

Dataset #2

In [30]:
# Dataset 2

df2 = pd.read_csv('data/2020-11-09_fc_R-80.csv',delimiter="\t", header=1).iloc[:-14,:2]

df2.columns=['Time','Voltage']
df2 = df2.stack().str.replace(',','.').unstack()  
df2 = df2.fillna(0)

df2['Time'] = pd.to_datetime(df2['Time'], format='%d.%m.%Y %H:%M:%S.%f')

for col in df2.columns[1:]:
    df2[col]=df2[col].astype(float)/10

Try if data is formatted correctly:

In [31]:
pd.concat([df1, df2]).sort_values('Time').reset_index(drop=True)

,Time,Vmon.U1,Voltage
0,2020-11-09 11:55:00,NaN,0.8663
1,2020-11-09 11:55:01,NaN,0.8663
2,2020-11-09 11:55:02,NaN,0.8663
3,2020-11-09 11:55:03,NaN,0.8663
4,2020-11-09 11:55:04,NaN,0.8663
...,...,...,...
30490,2020-11-09 19:54:56,NaN,0.8695
30491,2020-11-09 19:54:57,NaN,0.8695
30492,2020-11-09 19:54:58,NaN,0.8695
30493,2020-11-09 19:54:59,NaN,0.8695


Merge data

In [32]:
merge_list = [df1, df2]
merge_on   = 'Time'

In [33]:
def merger(merge_list,merge_on,data=[]):

    merge1 = merge_list[0].merge(merge_list[1], how='left', on=merge_on)
    merge2 = merge_list[1].merge(merge_list[0], how='left', on=merge_on)

    data = pd.concat([merge1,merge2]).sort_values(merge_on)
    data = data.drop_duplicates(subset=[merge_on]).reset_index(drop=True)

    if len(merge_list) > 2:
        for extra_merge in range(2,len(merge_list)):
            merge1 = data.merge(merge_list[extra_merge], how='left', on=merge_on)
            merge2 = merge_list[extra_merge].merge(data, how='left', on=merge_on)

            data = pd.concat([merge1,merge2]).sort_values(merge_on)
            data = data.drop_duplicates(subset=[merge_on]).reset_index(drop=True)
    
    return data
    
data = merger(merge_list,merge_on)
data

,Time,Vmon.U1,Voltage
0,2020-11-09 11:55:00,NaN,0.8663
1,2020-11-09 11:55:01,NaN,0.8663
2,2020-11-09 11:55:02,NaN,0.8663
3,2020-11-09 11:55:03,NaN,0.8663
4,2020-11-09 11:55:04,NaN,0.8663
...,...,...,...
30489,2020-11-09 19:54:56,NaN,0.8695
30490,2020-11-09 19:54:57,NaN,0.8695
30491,2020-11-09 19:54:58,NaN,0.8695
30492,2020-11-09 19:54:59,NaN,0.8695


Interpolate

In [34]:
data_int1 = data.interpolate(limit=10, limit_direction='forward')
data_int1 = data_int1.dropna()

Plot

In [35]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=data_int1['Time'],y=data_int1.iloc[:,1],name='dataset 1',
                         mode='lines',line=dict(color='black')))

fig.add_trace(go.Scatter(x=data_int1['Time'],y=data_int1.iloc[:,2],name='dataset 2',
                         mode='lines',line=dict(color='blue')))
fig.show()